In [35]:
#from alpaca_functions import *
# import alpaca_functions
# import stonk_pull 
# import signals
import datetime
#from binance import *
#import binance
import json
from datetime import time
import time
from time import mktime
import hmac
import hashlib
import requests

In [71]:
# This simulates holding a current bank account that can send and receive
# money via ACH transfers to and from Alpaca
# We are assuming we have 100,000 in the bank and we are sending 100,000 to our alpaca algorithm

#We will be using STRIPE as our shared bank account, see stripe.py 

beginning_deposit = 100000

bank_balance = 100000

send_to_alpaca_amount = 100000

withdraw_from_alpaca_amount = 0

binance_balance = 0

profits = 0

In [72]:
#load alpaca api keys
api = stonk_pull.load()


C:\Users\costa\desktop\byte\rice\ru-hou-fin-pt-03-2020-u-c\Project3_Doh\Final\stonk_pull.py:16: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')


In [73]:
# Send money to alpaca. This function calculates a new bank balance after we send money to alpaca
# In reality this would be an ACH transfer from our bank to alpaca.
send_to_alpaca(send_to_alpaca_amount, bank_balance) 


Transfer to alpaca successful. Your bank balance is now 0 dollars



0

In [74]:
#Pull the data from alphavantage
data = stonk_pull.data_pull(api)
data.head()

,SPY Open,SPY High,SPY Low,SPY Close,SPY Adjusted Close,SPY Volume,SPY Dividend Amount,SPY Split Coefficient
date,,,,,,,,
2020-08-21,337.92,339.72,337.5500,339.48,339.48,55106628.0,0.0,1.0
2020-08-20,335.36,338.80,335.2200,338.28,338.28,42207826.0,0.0,1.0
2020-08-19,339.05,339.61,336.6200,337.23,337.23,68054244.0,0.0,1.0
2020-08-18,338.34,339.10,336.6100,338.64,338.64,38733908.0,0.0,1.0
2020-08-17,337.94,338.34,336.8517,337.91,337.91,34496002.0,0.0,1.0


In [75]:
#Clean the data
stock_data_df = stonk_pull.data_clean(data)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
2020-08-21,337.92,339.48,55106628.0
2020-08-20,335.36,338.28,42207826.0
2020-08-19,339.05,337.23,68054244.0
2020-08-18,338.34,338.64,38733908.0
2020-08-17,337.94,337.91,34496002.0


In [76]:
#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
1999-11-01,136.5000,135.5625,4006500.0
1999-11-02,135.9687,134.5937,6516900.0
1999-11-03,136.0000,135.5000,7222300.0
1999-11-04,136.7500,136.5312,7907500.0
1999-11-05,138.6250,137.8750,7431500.0


In [77]:
#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['SPY Close'].to_frame()
signals_df.head()

,SPY Close
date,
1999-11-01,135.5625
1999-11-02,134.5937
1999-11-03,135.5000
1999-11-04,136.5312
1999-11-05,137.8750


In [78]:
#This function generates the signals dataframe
signals.generate_sma2_sma13_signals(signals_df)

,SPY Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
1999-11-01,135.5625,NaN,NaN,0.0,NaN
1999-11-02,134.5937,135.07810,NaN,0.0,0.0
1999-11-03,135.5000,135.04685,NaN,0.0,0.0
1999-11-04,136.5312,136.01560,NaN,0.0,0.0
1999-11-05,137.8750,137.20310,NaN,0.0,0.0
...,...,...,...,...,...
2020-08-17,337.9100,337.37500,332.902308,1.0,0.0
2020-08-18,338.6400,338.27500,334.031538,1.0,0.0
2020-08-19,337.2300,337.93500,334.855385,1.0,0.0


In [79]:
#Send Buy and Sell orders to alpaca using the signal, 1 = buy, -1 = sell, 0 = hold

if signals_df['Buy/Sell'].iloc[-1] == 1.0:
    symbol = "SPY"
    alpaca_functions.buy(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == -1.0:
    symbol = "SPY"
    alpaca_functions.sell(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == 0.0:
    print("The signal is hold. No action taken")

print(f"Current positions are: {api.list_positions()}")

The signal is hold. No action taken
Current positions are: [Position({   'asset_class': 'us_equity',
    'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
    'avg_entry_price': '342.095',
    'change_today': '0.0072758336279015',
    'cost_basis': '202520.24',
    'current_price': '341.95',
    'exchange': 'ARCA',
    'lastday_price': '339.48',
    'market_value': '202434.4',
    'qty': '592',
    'side': 'long',
    'symbol': 'SPY',
    'unrealized_intraday_pl': '-85.84',
    'unrealized_intraday_plpc': '-0.0004238588696122',
    'unrealized_pl': '-85.84',
    'unrealized_plpc': '-0.0004238588696122'})]


In [80]:
#On the first of the month we will send our profits from alpaca to the bank

day_of_month = (date.today()).strftime("%d")

if day_of_month == '1':
    transfer_profits_from_alpaca(bank_balance, beginning_deposit)
else:
    print("No profits were transferred to the bank from alpaca today.")  
    



No profits were transferred to the bank from alpaca today.


In [81]:
#On the second day of the month we will send our profits from our bank to the binance cryptoexchange

day_of_month = (date.today()).strftime("%d")
bank_balance  = 130000
binance_balance = 0

#If the the day of the month is 2, we transfer profits to binance and recalculate the bank balance and binance balance

if day_of_month == '2':
    #transfer function returns a list with the new bank balance and the new binance balance
    balances = transfer_profits_to_exchange(bank_balance, binance_balance)
    
    #set the new bank and the new binance balances
    bank_balance = balances[0]
    binance_balance = balances[1]
    
else:
    print("No profits were transferred from the bank to binance today.")  

print("bank balance:", bank_balance)
print("binance balance:", binance_balance)

No profits were transferred from the bank to binance today.
bank balance: 130000
binance balance: 0


In [82]:
# Get price for ETH through binance api
eth_price = get_eth_price()
eth_price = float(eth_price)
eth_price

407.4

In [83]:
#Calculate number of ether to buy
eth_to_buy = binance_balance/eth_price
eth_to_buy


0.0

In [89]:
#This buys one share of ETH, need to figure out how to change qty in the function. 
#Need to figure out how to correctly submit timestamp and signature
buy_ETH()

b'{"code":-1022,"msg":"Signature for this request is not valid."}'


b'{"code":-1022,"msg":"Signature for this request is not valid."}'

In [90]:
#check account holdings, need to send proper signature
get_account_holdings()

b'{"code":-1022,"msg":"Signature for this request is not valid."}'


b'{"code":-1022,"msg":"Signature for this request is not valid."}'

In [86]:
#check recent trades to see if trade went through
check_recent_trades()


b'[{"id":0,"price":"386.26000000","qty":"1.00000000","quoteQty":"386.26000000","time":1596440526404,"isBuyerMaker":true,"isBestMatch":true},{"id":1,"price":"414.15000000","qty":"0.02714000","quoteQty":"11.24003100","time":1597851105665,"isBuyerMaker":true,"isBestMatch":true},{"id":2,"price":"414.15000000","qty":"0.02729000","quoteQty":"11.30215350","time":1597854349295,"isBuyerMaker":true,"isBestMatch":true},{"id":3,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854483275,"isBuyerMaker":true,"isBestMatch":true},{"id":4,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854687743,"isBuyerMaker":true,"isBestMatch":true},{"id":5,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854803719,"isBuyerMaker":true,"isBestMatch":true},{"id":6,"price":"414.15000000","qty":"0.02738000","quoteQty":"11.33942700","time":1597854865008,"isBuyerMaker":true,"isBestMatch":true},{"id":7,"price":"414.15000000","qty":"0.0273

b'[{"id":0,"price":"386.26000000","qty":"1.00000000","quoteQty":"386.26000000","time":1596440526404,"isBuyerMaker":true,"isBestMatch":true},{"id":1,"price":"414.15000000","qty":"0.02714000","quoteQty":"11.24003100","time":1597851105665,"isBuyerMaker":true,"isBestMatch":true},{"id":2,"price":"414.15000000","qty":"0.02729000","quoteQty":"11.30215350","time":1597854349295,"isBuyerMaker":true,"isBestMatch":true},{"id":3,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854483275,"isBuyerMaker":true,"isBestMatch":true},{"id":4,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854687743,"isBuyerMaker":true,"isBestMatch":true},{"id":5,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854803719,"isBuyerMaker":true,"isBestMatch":true},{"id":6,"price":"414.15000000","qty":"0.02738000","quoteQty":"11.33942700","time":1597854865008,"isBuyerMaker":true,"isBestMatch":true},{"id":7,"price":"414.15000000","qty":"0.0273

In [87]:
#After checking the balance we send the ether to the account address of our profitsplitter.
#The profitsplitter should automatically divide the profits and send ether to each team member's wallet.

address = '0xf7748a5b4B867293FeC8ec0c3d3f2dCAc29d7a2e'
withdraw()



b'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <title>Binance Spot Test Network</title>\n    <link rel="shortcut icon" type="image/x-icon" href="/static/images/common/favicon.ico"/>\n    <style>\nhtml {\n    margin: 0;\n}\n\nbody {\n    display: flex;\n    flex-direction: column;\n    margin: 0;\n    padding: 0;\n    min-height: 100vh;\n    font-family: BinancePlex, Arial, PingFangSC-Regular, \'Microsoft YaHei\', sans-serif !important;\n}\n\n#header {\n    height: 64px;\n    background-color: #12161C;\n    margin: 0;\n    padding: 0;\n}\n\n#body {\n    display: flex;\n    flex: 1;\n    margin: 0;\n    padding: 0;\n}\n\n#content {\n    width: 800px;\n    margin-left: auto;\n    margin-right: auto;\n    line-height: 26px;\n}\n\n#footer {\n    background-color: #12161C;\n    font-size: 12px;\n    line-height: 16px;\n    color: #76808F;\n    text-align: center;\n    margin-top: 20px;\n    margin-bottom: 0;\n    margin-left: 0;\n    margin-right: 0;\n    padding: 12px;\n}\n\n#logo {\n    

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <title>Binance Spot Test Network</title>\n    <link rel="shortcut icon" type="image/x-icon" href="/static/images/common/favicon.ico"/>\n    <style>\nhtml {\n    margin: 0;\n}\n\nbody {\n    display: flex;\n    flex-direction: column;\n    margin: 0;\n    padding: 0;\n    min-height: 100vh;\n    font-family: BinancePlex, Arial, PingFangSC-Regular, \'Microsoft YaHei\', sans-serif !important;\n}\n\n#header {\n    height: 64px;\n    background-color: #12161C;\n    margin: 0;\n    padding: 0;\n}\n\n#body {\n    display: flex;\n    flex: 1;\n    margin: 0;\n    padding: 0;\n}\n\n#content {\n    width: 800px;\n    margin-left: auto;\n    margin-right: auto;\n    line-height: 26px;\n}\n\n#footer {\n    background-color: #12161C;\n    font-size: 12px;\n    line-height: 16px;\n    color: #76808F;\n    text-align: center;\n    margin-top: 20px;\n    margin-bottom: 0;\n    margin-left: 0;\n    margin-right: 0;\n    padding: 12px;\n}\n\n#logo {\n    

In [88]:
 get_account_holdings()  

b'{"code":-1022,"msg":"Signature for this request is not valid."}'


b'{"code":-1022,"msg":"Signature for this request is not valid."}'

In [127]:
import urllib3
import urllib.request
import urllib.parse


def buy_ETH():
    
    '''Buy request for binance api, this will buy Ether'''
    
    secret = 'NhqPtmdSJYdKjVHjA7PZj4Mge3R5YNiP1e3UZjInClVN65XAbvqqM6A7H5fATj0j'
    
    #Get timestamp in milliseconds 
    timestamp = int(time.time()*1000)
  
    #timestamp_hashed = hmac.new(secret.encode('utf-8'),(timestamp.encode('utf-8'), hashlib.sha256).hexdigest())
   
    #separate parameters from url
    query_string = f"symbol=ETHBUSD&side=BUY&type=MARKET&quantity=1&newClientOrderId=my_order_id_1&newOrderRespType=ACK&timestamp={timestamp}" 
    query_string = urllib.parse.quote(querystring)
    
     #use secret key and query string to create signature

    signature = hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
   
    url = f"https://testnet.binance.vision/api/v3/order?symbol=ETHBUSD&side=BUY&type=MARKET&quantity=1&newClientOrderId=my_order_id_1&newOrderRespType=ACK&timestamp={timestamp}&signature={signature}" 

    payload = {}
    headers = {
       'Content-Type': 'application/json',
       'X-MBX-APIKEY': 'IfBnaRxLujIJRXGkIgNq2z3g1F8QC9kA59sJ6xRPObURqdCE5jdGDpp8kFv72YcU'
     }

    response = requests.request("POST", url, headers=headers, data = payload)
    #print(response.text.encode('utf8'))
    return((response.text.encode('utf8')))

buy_ETH()

b'{"code":-1022,"msg":"Signature for this request is not valid."}'

In [125]:
import urllib3
import urllib.request
import urllib.parse


api_key = 'IfBnaRxLujIJRXGkIgNq2z3g1F8QC9kA59sJ6xRPObURqdCE5jdGDpp8kFv72YcU'
api_secret = 'NhqPtmdSJYdKjVHjA7PZj4Mge3R5YNiP1e3UZjInClVN65XAbvqqM6A7H5fATj0j'
request_url = 'https://api.binance.com/api/v3/openOrders?'
timestamp = int(time.time() * 1000)
symbol = 'BNBBTC'
querystring = '{symbol : symbol, timestamp : timestamp}'
querystring = urllib.parse.quote(querystring)
signature = hmac.new(api_secret.encode('utf-8'), querystring.encode('utf-8'), hashlib.sha256).hexdigest()
request_url += querystring + '&signature=' + signature
r = requests.get(request_url, headers={"X-MBX-APIKEY": api_key})
response = requests.request("POST", url, headers=headers, data=)

'ac411cefa4b695a3298c64f23365490e534e9c558cbe2c5f7f6bf3dff25ec853'

In [42]:
import urllib3
import urllib.request
import urllib.parse

def buy_ETH():
    
    api_key = 'IfBnaRxLujIJRXGkIgNq2z3g1F8QC9kA59sJ6xRPObURqdCE5jdGDpp8kFv72YcU'
    secret = 'NhqPtmdSJYdKjVHjA7PZj4Mge3R5YNiP1e3UZjInClVN65XAbvqqM6A7H5fATj0j'
    request_url = 'https://api.binance.com/api/v3/openOrders?'
    timestamp = int(time.time() * 1000)
    symbol = 'BNBBTC'
    querystring = urlencode({'symbol' : symbol, 'timestamp' : timestamp})
    #querystring = urllib.parse.quote(querystring)
    signature = hmac.new(secret.encode('utf-8'), querystring.encode('utf-8'), hashlib.sha256).hexdigest()
    request_url += querystring + '&signature=' + signature
    payload = {}
    headers = {
       'Content-Type': 'application/json',
       'X-MBX-APIKEY': 'IfBnaRxLujIJRXGkIgNq2z3g1F8QC9kA59sJ6xRPObURqdCE5jdGDpp8kFv72YcU'
     }
    r = requests.get(request_url, headers=headers)
    response = requests.request("POST", request_url, headers=headers, data=payload)
    
    return(response)


In [43]:
buy_ETH()

NameError: name 'urlencode' is not defined

In [22]:
datetime.time()

datetime.time(0, 0)

In [25]:
time.time()

1598303156.0367098

In [51]:
from binance.client import Client

client.ping()

{}

In [54]:
client = Client('ehalnzjFpYi1lIdsGu4236ddOMMypbAyqjMH3FS7XkYNagISeZQGtiyne9hcmrhz', 'OaMImZjZCv5JHDqRIyaJjYEvhBPdGhUlsKVWN613gPdBMIdFjdnoF4tB98O16BMY')
client.get_exchange_info()
# order = client.create_test_order(
#     symbol='BNBBTC',
#     side=Client.SIDE_BUY,
#     type=Client.ORDER_TYPE_MARKET,
#     quantity=100,
# )

{'timezone': 'UTC',
 'serverTime': 1598319007982,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 100},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 200000}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'isSpotTradingAllowed': True,
   'isMarginTradingAllowed': True,
   'filters': [{'filterType': 'PRICE_FILTER',
     'minPrice': '0.00000100',
     'maxPrice': '10000